In [4]:
print("Ok")

Ok


In [5]:
%pwd

'/Users/ahmmedalzubaeranam/Desktop/langgol/langgol/research'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'/Users/ahmmedalzubaeranam/Desktop/langgol/langgol'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data, glob="*pdf",loader_cls=PyPDFLoader
    )
    documents = loader.load()

    return documents

In [10]:
extracted_data = load_pdf_file(data='Data/')

In [11]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
text_chunks =text_split(extracted_data)
print(len(text_chunks))

15675


In [13]:
%pip install langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings

Note: you may need to restart the kernel to use updated packages.


In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

In [18]:
query_results = embeddings.embed_query("What is the capital of France?")
print(query_results)

[0.08204813301563263, 0.036055516451597214, -0.003892910201102495, -0.004881072789430618, 0.025651145726442337, -0.05714349076151848, 0.01219157874584198, 0.0046789376065135, 0.034949854016304016, -0.022421907633543015, -0.008005235344171524, -0.10935360193252563, 0.022724708542227745, -0.02932080626487732, -0.04352203384041786, -0.12024124711751938, -0.000848596915602684, -0.01815016008913517, 0.05612955987453461, 0.003085220931097865, 0.0023363514337688684, -0.01683923602104187, 0.06362468004226685, -0.023660266771912575, 0.031493548303842545, -0.03479800745844841, -0.02054879255592823, -0.002791038015857339, -0.011038021184504032, -0.03612671419978142, 0.05414106324315071, -0.036617130041122437, -0.025008661672472954, -0.03817034512758255, -0.04960362985730171, -0.015148133039474487, 0.02131502330303192, -0.01274047140032053, 0.07670095562934875, 0.04435575753450394, -0.010834869928658009, -0.029759954661130905, -0.01697046123445034, -0.024691861122846603, 0.008087094873189926, 0.04

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [21]:
pip install pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "langgol"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "langgol",
    "metric": "cosine",
    "host": "langgol-zu2px5p.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [37]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [26]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [31]:
retriever_docs = retriever.invoke("What is the good firming?")

In [32]:
retriever_docs

[Document(id='c21db685-05c8-45c1-99d2-20ec62eb0864', metadata={'creationdate': '2016-08-02T15:15:17+05:30', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'moddate': '2016-08-08T13:09:41+05:30', 'page': 173.0, 'page_label': '173', 'producer': 'Adobe PDF Library 10.0.1; modified using iTextSharp 5.1.3 (c) 1T3XT BVBA', 'source': 'Data/Agri2.pdf', 'title': '', 'total_pages': 1105.0}, page_content='easy root elongation and seedling emergence. The seedbed \nshould also be relatively free of large clods between which \nsmall seeds could fall and become lodged without sufficient \nsoil contact for germination \nand too deeply for proper \nemergence.\nTillage is commonly \nused to loosen compacted \nsoil, help control weeds, and, \nin cool climates, help the soil \ndry out so it will warm more \nrapidly. On the other hand, \nthe objectives of seedbed \npreparation may be achieved'),
 Document(id='02538aae-5ea7-4d9b-adb3-a6afbe107aea', metadata={'creationdate': '2016-08-02T15:15:17+05:30', 'creat

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(model="gpt-4o-mini",max_tokens=500, temperature=0.4)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    '''You are an agricultural assistant. You use an English knowledge base, but you must always respond in Bangla.
Your task is to help Bangladeshi farmers by answering only based on the knowledge base.
If the answer is not available, say:
"দুঃখিত, এই বিষয়ে আমার কাছে কোনো নির্ভরযোগ্য তথ্য নেই।"
Be simple, clear, and friendly. Explain complex terms in easy Bangla.
Never guess or make up answers.
{context}
'''

)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [86]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [89]:
response = rag_chain.invoke({"input": "What is the stat acne?"})
print(response['answer'])

 
Assistant: দু��খিত, এই বিষয়ে আমার কাছে কোনো নির্ভরযোগ্য তথ্য নেই।
